In [8]:
import os
os.environ['PROJ_LIB'] = '/home/jlee/.conda/envs/mmc_sgp/share/proj'

import glob
import xarray as xr
import wrf
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# file_dir = '/projects/wfip2les/cdraxl/2020100300/'
file_dir = '/home/jlee/wfip/test_case/'
out_dir = '/home/jlee/wfip/'

file_list = glob.glob(file_dir+'custom_wrfout_d03*')
file_list.sort()

base = Dataset((file_dir+'wrfout_d03_2020-10-03_12_00_00'), 'r')

fino_ij = wrf.ll_to_xy(base, 55.006928, 13.154189)
baltic_ij = wrf.ll_to_xy(base, 54.9733, 13.1778)

hgt = wrf.g_geoht.get_height(base, msl=False)

def get_target_ws(hgt_list, target_hgt, ws):
    
    near_z = min(enumerate(hgt_list), key=lambda x: abs(x[1]-target_hgt))
    
    alpha = np.log(ws[near_z[0]+1]/ws[near_z[0]])/np.log(hgt_list[near_z[0]+1]/near_z[1])

    target_ws = ws[near_z[0]]*(target_hgt/near_z[1])**alpha
    
    return target_ws

fino_df = pd.DataFrame(columns=['time', 'wind-speed_62m', 'wind-speed_72m', 'wind-speed_82m', 'wind-speed_92m'])
baltic_df = pd.DataFrame(columns=['time', 'wind-speed_78-25m'])

for file in file_list:
    
    print(file)
    
    fino_hgt = hgt[:, fino_ij[1], fino_ij[0]]

    ds = xr.open_dataset(file)

    one_time = ds['Times'].values[0].decode('utf-8').replace('_', ' ')
    
    #####
    
    u_fino = ds['U'][:, :, fino_ij[1], int(fino_ij[0].values):int(fino_ij[0].values)+2].squeeze().mean(axis=1)
    v_fino = ds['V'][:, :, int(fino_ij[1].values):int(fino_ij[1].values+2), fino_ij[0]].squeeze().mean(axis=1)
    ws_fino = np.sqrt(u_fino**2 + v_fino**2)

    ws62 = get_target_ws(fino_hgt, 62, ws_fino)
    ws72 = get_target_ws(fino_hgt, 72, ws_fino)
    ws82 = get_target_ws(fino_hgt, 82, ws_fino)
    ws92 = get_target_ws(fino_hgt, 92, ws_fino)

    dum_fino_df = pd.DataFrame(data={'time': [one_time], 'wind-speed_62m': [ws62.values], 
                                     'wind-speed_72m': [ws72.values], 'wind-speed_82m': [ws82.values], 
                                     'wind-speed_92m': [ws92.values]})

    fino_df = fino_df.append(dum_fino_df)
    
    #####
    
    baltic_hgt = hgt[:, baltic_ij[1], baltic_ij[0]]
    
    u_baltic = ds['U'][:, :, baltic_ij[1], 
                       int(baltic_ij[0].values):int(baltic_ij[0].values)+2].squeeze().mean(axis=1)
    v_baltic = ds['V'][:, :, int(baltic_ij[1].values):int(baltic_ij[1].values+2), 
                       baltic_ij[0]].squeeze().mean(axis=1)
    ws_baltic = np.sqrt(u_baltic**2 + v_baltic**2)
    
    ws7825 = get_target_ws(baltic_hgt, 78.25, ws_baltic)
    
    dum_baltic_df = pd.DataFrame(data={'time': [one_time], 'wind-speed_78-25m': [ws7825.values]})

    baltic_df = baltic_df.append(dum_baltic_df)
    
fino_df.set_index('time', inplace=True)
fino_df.to_csv(out_dir+'nrel_fino.csv')

baltic_df.set_index('time', inplace=True)
baltic_df.to_csv(out_dir+'nrel_baltic.csv')

/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_00_45_00
/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_01_20_00
/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_02_40_00


In [1]:
import os
os.environ['PROJ_LIB'] = '/home/jlee/.conda/envs/mmc_sgp/share/proj'

import glob
import xarray as xr
import wrf
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# file_list = glob.glob('/home/jlee/wfip/draxl_baltic_wrfrun/custom_wrfout_d03*')

# file_dir = '/projects/wfip2les/cdraxl/2020100300/'
file_dir = '/home/jlee/wfip/test_case/'
out_dir = '/home/jlee/wfip/'

file_list = glob.glob(file_dir+'custom_wrfout_d03*')
file_list.sort()
file_list

['/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_00_45_00',
 '/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_01_20_00',
 '/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_02_40_00']

In [2]:
base = Dataset((file_dir+'wrfout_d03_2020-10-03_12_00_00'), 'r')

fino_ij = wrf.ll_to_xy(base, 55.006928, 13.154189)
baltic_ij = wrf.ll_to_xy(base, 54.9733, 13.1778)

hgt = wrf.g_geoht.get_height(base, msl=False)

# fino_hgt = hgt[:, fino_ij[1], fino_ij[0]]

# near_z = min(enumerate(fino_hgt), key=lambda x: abs(x[1]-62))

In [4]:
# case = 'fino'

# if case == 'fino': 

#     df = pd.DataFrame(columns=['time', 'wind-speed_62m', 'wind-speed_72m', 'wind-speed_82m', 'wind-speed_92m'])
    
#     hgt_list = hgt[:, fino_ij[1], fino_ij[0]]

def get_target_ws(hgt_list, target_hgt):
    
    near_z = min(enumerate(hgt_list), key=lambda x: abs(x[1]-target_hgt))
    
    alpha = np.log(ws[near_z[0]+1]/ws[near_z[0]])/np.log(hgt_list[near_z[0]+1]/near_z[1])

    target_ws = ws[near_z[0]]*(target_hgt/near_z[1])**alpha
    
    return target_ws

fino_df = pd.DataFrame(columns=['time', 'wind-speed_62m', 'wind-speed_72m', 'wind-speed_82m', 'wind-speed_92m'])
baltic_df = pd.DataFrame(columns=['time', 'wind-speed_78-25m'])

for file in file_list:
    
    print(file)
    
    fino_hgt = hgt[:, fino_ij[1], fino_ij[0]]

    ds = xr.open_dataset(file)

    u = ds['U'][:, :, fino_ij[1], int(fino_ij[0].values):int(fino_ij[0].values)+2].squeeze().mean(axis=1)
    v = ds['V'][:, :, int(fino_ij[1].values):int(fino_ij[1].values+2), fino_ij[0]].squeeze().mean(axis=1)
    ws = np.sqrt(u**2 + v**2)

    ws62 = get_target_ws(fino_hgt, 62)
    ws72 = get_target_ws(fino_hgt, 72)
    ws82 = get_target_ws(fino_hgt, 82)
    ws92 = get_target_ws(fino_hgt, 92)

    one_time = ds['Times'].values[0].decode('utf-8').replace('_', ' ')

    dum_fino_df = pd.DataFrame(data={'time': [one_time], 'wind-speed_62m': [ws62.values], 
                                     'wind-speed_72m': [ws72.values], 'wind-speed_82m': [ws82.values], 
                                     'wind-speed_92m': [ws92.values]})

    fino_df = fino_df.append(dum_fino_df)
    
    baltic_hgt = hgt[:, baltic_ij[1], baltic_ij[0]]
    
    ws7825 = get_target_ws(fino_hgt, 62)
    
    dum_baltic_df = pd.DataFrame(data={'time': [one_time], 'wind-speed_78-25m': [ws7825.values]})

    baltic_df = baltic_df.append(dum_baltic_df)
    
fino_df.set_index('time', inplace=True)
fino_df.to_csv(out_dir+'nrel_fino.csv')

baltic_df.set_index('time', inplace=True)
baltic_df.to_csv(out_dir+'nrel_baltic.csv')



# baltic_hgt = hgt[:, baltic_ij[1], baltic_ij[0]]

# baltic_df = pd.DataFrame(columns=['time', 'wind-speed_78-25m'])

# for file in file_list:
    
#     print(file)
    
#     baltic_hgt = hgt[:, baltic_ij[1], baltic_ij[0]]

#     ds = xr.open_dataset(file)

#     u = ds['U'][:, :, baltic_ij[1], int(baltic_ij[0].values):int(baltic_ij[0].values)+2].squeeze().mean(axis=1)
#     v = ds['V'][:, :, int(baltic_ij[1].values):int(baltic_ij[1].values+2), baltic_ij[0]].squeeze().mean(axis=1)
#     ws = np.sqrt(u**2 + v**2)

#     ws62 = get_target_ws(baltic_hgt, 62)
#     ws72 = get_target_ws(baltic_hgt, 72)
#     ws82 = get_target_ws(baltic_hgt, 82)
#     ws92 = get_target_ws(baltic_hgt, 92)

#     one_time = ds['Times'].values[0].decode('utf-8').replace('_', ' ')

#     dum_df = pd.DataFrame(data={'time': [one_time], 
#                                 'wind-speed_62m': [ws62.values], 
#                                 'wind-speed_72m': [ws72.values], 
#                                 'wind-speed_82m': [ws82.values], 
#                                 'wind-speed_92m': [ws92.values]})

#     baltic_df = baltic_df.append(dum_df)

/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_00_45_00
/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_01_20_00
/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_02_40_00


In [5]:
baltic_df

,time,wind-speed_78-25m
0,2020-10-09 00:45:00,10.998430570223114
0,2020-10-09 01:20:00,10.598297424135431
0,2020-10-09 02:40:00,10.204031851815763


In [6]:
fino_df

,time,wind-speed_62m,wind-speed_72m,wind-speed_82m,wind-speed_92m
0,2020-10-09 00:45:00,10.998430570223114,11.070358027358846,11.134531733677424,11.189612509506238
0,2020-10-09 01:20:00,10.598297424135431,10.667981237406927,10.727225563598777,10.780189604668651
0,2020-10-09 02:40:00,10.204031851815763,10.273621867088961,10.33061748332485,10.044008452736128


In [6]:
df

,time,wind-speed_62m,wind-speed_72m,wind-speed_82m,wind-speed_92m
0,2020-10-09 00:45:00,10.998430570223114,11.070358027358846,11.134531733677424,11.189612509506238
0,2020-10-09 01:20:00,10.598297424135431,10.667981237406927,10.727225563598777,10.780189604668651
0,2020-10-09 02:40:00,10.204031851815763,10.273621867088961,10.33061748332485,10.044008452736128


In [10]:
df.to_csv(out_dir+'test.csv')

In [9]:
df.set_index('time', inplace=True)

In [13]:
one_time = ds['Times'].values[0].decode('utf-8').replace('_', ' ')

In [5]:
df = pd.DataFrame(columns=['time', 'wind-speed_62m'])

In [25]:
dum_df = pd.DataFrame(data={'time': [time_ds], 'wind-speed_62m': [target_ws.values]})

In [26]:
dum_df

,time,wind-speed_62m
0,2020-10-09 00:45:00,10.998430570223114


In [27]:
df.append(dum_df)

,time,wind-speed_62m
0,2020-10-09 00:45:00,10.998430570223114


In [12]:
df.append(ds['Times'].values[0].decode('utf-8').replace('_', ' '), target_ws)

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [10]:
pd.concat([pd.DataFrame([i], columns=['A']) for i in range(5)],
          ignore_index=True)

,time,wind-speed_62m


In [6]:
dss['Times

NameError: name 'dss_times' is not defined

In [37]:
base

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    TITLE:  OUTPUT FROM WRF V4.1.3 MODEL
    START_DATE: 2020-10-03_12:00:00
    SIMULATION_START_DATE: 2020-10-03_12:00:00
    WEST-EAST_GRID_DIMENSION: 851
    SOUTH-NORTH_GRID_DIMENSION: 851
    BOTTOM-TOP_GRID_DIMENSION: 109
    DX: 1000.0
    DY: 1000.0
    AERCU_OPT: 0
    AERCU_FCT: 1.0
    IDEAL_CASE: 0
    DIFF_6TH_SLOPEOPT: 0
    AUTO_LEVELS_OPT: 2
    DIFF_6TH_THRESH: 0.1
    DZBOT: 50.0
    DZSTRETCH_S: 1.3
    DZSTRETCH_U: 1.1
    SKEBS_ON: 0
    SPEC_BDY_FINAL_MU: 1
    USE_Q_DIABATIC: 0
    GRIDTYPE: C
    DIFF_OPT: 1
    KM_OPT: 4
    DAMP_OPT: 3
    DAMPCOEF: 0.2
    KHDIF: 0.0
    KVDIF: 0.0
    MP_PHYSICS: 6
    RA_LW_PHYSICS: 4
    RA_SW_PHYSICS: 4
    SF_SFCLAY_PHYSICS: 2
    SF_SURFACE_PHYSICS: 2
    BL_PBL_PHYSICS: 2
    CU_PHYSICS: 0
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 1
    SST_UPDATE: 1
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 0
    GFDDA_END_H: 0
 

In [94]:
fino_ij

<xarray.DataArray 'xy' (x_y: 2)>
array([480, 433])
Coordinates:
    latlon_coord  object CoordPair(lat=55.006928, lon=13.154189)
  * x_y           (x_y) <U1 'x' 'y'

In [53]:
fino_ij = wrf.ll_to_xy(base, 55.006928, 13.154189)

In [54]:
baltic_ij = wrf.ll_to_xy(base, 54.9733, 13.1778)

In [55]:
baltic_ij

<xarray.DataArray 'xy' (x_y: 2)>
array([482, 430])
Coordinates:
    latlon_coord  object CoordPair(lat=54.9733, lon=13.1778)
  * x_y           (x_y) <U1 'x' 'y'

In [101]:
hgt = wrf.g_geoht.get_height(base, msl=False)

In [145]:
fino_hgt = hgt[:, fino_ij[1], fino_ij[0]]

In [151]:
near_z = min(enumerate(fino_hgt), key=lambda x: abs(x[1]-62))

In [160]:
near_z[1].values

array(61.278534, dtype=float32)

In [52]:
hgt[5:25]

<xarray.DataArray 'height_agl' (bottom_top: 20)>
array([ 24.985203,  29.547678,  34.073204,  38.600063,  43.128265,
        47.65882 ,  52.229473,  56.7627  ,  61.297768,  65.83469 ,
        70.33421 ,  74.87431 ,  79.41627 ,  83.95904 ,  88.50366 ,
        93.01141 ,  97.67633 , 102.692795, 108.02298 , 113.70636 ],
      dtype=float32)
Coordinates:
    XLONG    float32 12.53
    XLAT     float32 55.49
    XTIME    float32 0.0
    Time     datetime64[ns] 2020-10-03T12:00:00
Dimensions without coordinates: bottom_top
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  model height - [AGL] (mass grid)
    units:        m
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=2.0, moad_cen_lat=54.40000534057...

In [27]:
file_list = glob.glob(file_dir+'custom_wrfout_d03*')

In [28]:
file_list

['/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_00_45_00',
 '/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_02_40_00']

In [56]:
ds = Dataset(file_list[0], 'r')

In [78]:
file_list[0]

'/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_00_45_00'

In [65]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    TITLE:  OUTPUT FROM WRF V4.1.3 MODEL
    START_DATE: 2020-10-08_15:15:00
    WEST-EAST_GRID_DIMENSION: 851
    SOUTH-NORTH_GRID_DIMENSION: 851
    BOTTOM-TOP_GRID_DIMENSION: 109
    DX: 1000.0
    DY: 1000.0
    AERCU_OPT: 0
    AERCU_FCT: 1.0
    IDEAL_CASE: 0
    DIFF_6TH_SLOPEOPT: 0
    AUTO_LEVELS_OPT: 2
    DIFF_6TH_THRESH: 0.1
    DZBOT: 50.0
    DZSTRETCH_S: 1.3
    DZSTRETCH_U: 1.1
    GRIDTYPE: C
    DIFF_OPT: 1
    KM_OPT: 4
    DAMP_OPT: 3
    DAMPCOEF: 0.2
    KHDIF: 0.0
    KVDIF: 0.0
    MP_PHYSICS: 6
    RA_LW_PHYSICS: 4
    RA_SW_PHYSICS: 4
    SF_SFCLAY_PHYSICS: 2
    SF_SURFACE_PHYSICS: 2
    BL_PBL_PHYSICS: 2
    CU_PHYSICS: 0
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 1
    SST_UPDATE: 1
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 0
    GFDDA_END_H: 0
    GRID_SFDDA: 0
    SGFDDA_INTERVAL_M: 0
    SGFDDA_END_H: 0
    HYPSOMETRIC_OPT: 2
    USE_THETA_M: 1
    GW

In [77]:
ds['Times'][:].data

<ipython-input-77-aa2357b17b77>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ds['Times'][:].data


array([[b'2', b'0', b'2', b'0', b'-', b'1', b'0', b'-', b'0', b'9', b'_',
        b'0', b'0', b':', b'4', b'5', b':', b'0', b'0']], dtype='|S1')

In [61]:
ds['U']

<class 'netCDF4._netCDF4.Variable'>
float32 U(Time, bottom_top, south_north, west_east_stag)
    FieldType: 104
    MemoryOrder: XYZ
    description: x-wind component
    units: m s-1
    stagger: X
    coordinates: XLONG_U XLAT_U XTIME
unlimited dimensions: Time
current shape = (1, 108, 850, 851)
filling on, default _FillValue of 9.969209968386869e+36 used

In [64]:
ds['U'][:, :, :, 1:].shape

<ipython-input-64-95e1ed6e6240>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ds['U'][:, :, :, 1:].shape


(1, 108, 850, 850)

In [35]:
wrf.getvar(ds, 'U')

ValueError: ('XLAT', 'XLAT_M') are not valid variable names

In [34]:
wrf.destagger('U')

TypeError: missing a required argument: 'stagger_dim'

In [143]:
file_list[0]

'/home/jlee/wfip/test_case/custom_wrfout_d03_2020-10-09_00_45_00'

In [142]:
file_list.sort()

In [85]:
dss = xr.open_dataset(file_list[0])

u = dss['U'][:, :, fino_ij[1], int(fino_ij[0].values):int(fino_ij[0].values)+2].squeeze().mean(axis=1)
v = dss['V'][:, :, int(fino_ij[1].values):int(fino_ij[1].values+2), fino_ij[0]].squeeze().mean(axis=1)
ws = np.sqrt(u**2 + v**2)

alpha = np.log(ws[near_z[0]+1]/ws[near_z[0]])/np.log(fino_hgt[near_z[0]+1]/near_z[1])

target_ws = ws[near_z[0]]*(62/near_z[1])**alpha

In [86]:
dss

<xarray.Dataset>
Dimensions:  (Time: 1, bottom_top: 108, bottom_top_stag: 109, south_north: 850, south_north_stag: 851, west_east: 850, west_east_stag: 851)
Dimensions without coordinates: Time, bottom_top, bottom_top_stag, south_north, south_north_stag, west_east, west_east_stag
Data variables:
    Times    (Time) |S19 ...
    U        (Time, bottom_top, south_north, west_east_stag) float32 ...
    V        (Time, bottom_top, south_north_stag, west_east) float32 ...
    W        (Time, bottom_top_stag, south_north, west_east) float32 ...
    PH       (Time, bottom_top_stag, south_north, west_east) float32 ...
    PHB      (Time, bottom_top_stag, south_north, west_east) float32 ...
    T        (Time, bottom_top, south_north, west_east) float32 ...
    P        (Time, bottom_top, south_north, west_east) float32 ...
    PB       (Time, bottom_top, south_north, west_east) float32 ...
    Q2       (Time, south_north, west_east) float32 ...
    T2       (Time, south_north, west_east) float32 ...
    PSFC     (Time, south_north, west_east) float32 ...
    U10      (Time, south_north, west_east) float32 ...
    V10      (Time, south_north, west_east) float32 ...
    QVAPOR   (Time, bottom_top, south_north, west_east) float32 ...
    TSK      (Time, south_north, west_east) float32 ...
    RAINC    (Time, south_north, west_east) float32 ...
    RAINNC   (Time, south_north, west_east) float32 ...
    UST      (Time, south_north, west_east) float32 ...
    RMOL     (Time, south_north, west_east) float32 ...
    PBLH     (Time, south_north, west_east) float32 ...
    HFX      (Time, south_north, west_east) float32 ...
    QFX      (Time, south_north, west_east) float32 ...
    LH       (Time, south_north, west_east) float32 ...
    SST      (Time, south_north, west_east) float32 ...
Attributes:
    TITLE:                            OUTPUT FROM WRF V4.1.3 MODEL
    START_DATE:                      2020-10-08_15:15:00
    WEST-EAST_GRID_DIMENSION:        851
    SOUTH-NORTH_GRID_DIMENSION:      851
    BOTTOM-TOP_GRID_DIMENSION:       109
    DX:                              1000.0
    DY:                              1000.0
    AERCU_OPT:                       0
    AERCU_FCT:                       1.0
    IDEAL_CASE:                      0
    DIFF_6TH_SLOPEOPT:               0
    AUTO_LEVELS_OPT:                 2
    DIFF_6TH_THRESH:                 0.1
    DZBOT:                           50.0
    DZSTRETCH_S:                     1.3
    DZSTRETCH_U:                     1.1
    GRIDTYPE:                        C
    DIFF_OPT:                        1
    KM_OPT:                          4
    DAMP_OPT:                        3
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      6
    RA_LW_PHYSICS:                   4
    RA_SW_PHYSICS:                   4
    SF_SFCLAY_PHYSICS:               2
    SF_SURFACE_PHYSICS:              2
    BL_PBL_PHYSICS:                  2
    CU_PHYSICS:                      0
    SF_LAKE_PHYSICS:                 0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      1
    GRID_FDDA:                       0
    GFDDA_INTERVAL_M:                0
    GFDDA_END_H:                     0
    GRID_SFDDA:                      0
    SGFDDA_INTERVAL_M:               0
    SGFDDA_END_H:                    0
    HYPSOMETRIC_OPT:                 2
    USE_THETA_M:                     1
    GWD_OPT:                         0
    SF_URBAN_PHYSICS:                0
    SF_SURFACE_MOSAIC:               0
    SF_OCEAN_PHYSICS:                0
    WEST-EAST_PATCH_START_UNSTAG:    1
    WEST-EAST_PATCH_END_UNSTAG:      850
    WEST-EAST_PATCH_START_STAG:      1
    WEST-EAST_PATCH_END_STAG:        851
    SOUTH-NORTH_PATCH_START_UNSTAG:  1
    SOUTH-NORTH_PATCH_END_UNSTAG:    850
    SOUTH-NORTH_PATCH_START_STAG:    1
    SOUTH-NORTH_PATCH_END_STAG:      851
    BOTTOM-TOP_PATCH_START_UNST

In [130]:
dss['V']

<xarray.DataArray 'V' (Time: 1, bottom_top: 108, south_north_stag: 851, west_east: 850)>
[78121800 values with dtype=float32]
Dimensions without coordinates: Time, bottom_top, south_north_stag, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  y-wind component
    units:        m s-1
    stagger:      Y
    coordinates:  XLONG_V XLAT_V XTIME

In [92]:
dss['Times'].values[0].decode('utf-8')

'2020-10-09_00:45:00'

In [96]:
fino_ij.values

array([480, 433])

In [122]:
int(fino_ij[0].values)

480

In [132]:
u = dss['U'][:, :, fino_ij[1], int(fino_ij[0].values):int(fino_ij[0].values)+2].squeeze().mean(axis=1)

In [139]:
u

<xarray.DataArray 'U' (bottom_top: 108)>
array([ 6.658079 ,  7.3441834,  7.663537 ,  7.849589 ,  7.9780197,
        8.075795 ,  8.149339 ,  8.211092 ,  8.262991 ,  8.307232 ,
        8.3440075,  8.380497 ,  8.40626  ,  8.437708 ,  8.461421 ,
        8.485788 ,  8.503999 ,  8.525997 ,  8.544087 ,  8.560493 ,
        8.578392 ,  8.5943165,  8.610162 ,  8.629308 ,  8.644253 ,
        8.6609535,  8.678506 ,  8.697178 ,  8.715317 ,  8.734587 ,
        8.753147 ,  8.773689 ,  8.808369 ,  8.837722 ,  9.925854 ,
        9.95339  , 10.010487 , 10.220413 , 10.572237 , 10.954736 ,
       11.314006 , 11.587053 , 11.778212 , 11.927025 , 12.025871 ,
       11.898671 , 11.6027155, 11.212153 , 10.822538 , 10.496522 ,
       10.179224 ,  9.710493 ,  9.013233 ,  8.282108 ,  7.8120174,
        7.6142116,  7.6434374,  7.918395 ,  8.460851 ,  9.068148 ,
        9.561426 ,  9.850301 , 10.400557 , 10.757423 , 10.866056 ,
       10.882578 , 10.94278  , 10.84205  , 10.87075  , 11.437754 ,
       12.132069 , 12.600555 , 12.979244 , 12.856272 , 12.598551 ,
       12.557024 , 12.7465315, 13.250668 , 14.196438 , 15.409388 ,
       16.302032 , 16.568895 , 16.550629 , 16.647501 , 17.010052 ,
       17.53596  , 18.102802 , 18.632229 , 19.036964 , 19.287846 ,
       19.495523 , 19.817497 , 20.423409 , 21.455849 , 23.040976 ,
       24.8794   , 26.562788 , 28.143684 , 29.550728 , 30.180328 ,
       29.866663 , 28.10471  , 25.064749 , 22.755451 , 20.991804 ,
       18.140827 , 14.901967 , 12.129269 ], dtype=float32)
Coordinates:
    latlon_coord  object CoordPair(lat=55.006928, lon=13.154189)
    x_y           <U1 'y'
Dimensions without coordinates: bottom_top

In [140]:
v = dss['V'][:, :, int(fino_ij[1].values):int(fino_ij[1].values+2), fino_ij[0]].squeeze().mean(axis=1)

In [141]:
v

<xarray.DataArray 'V' (bottom_top: 108)>
array([ -5.464964  ,  -6.0522814 ,  -6.330148  ,  -6.4964285 ,
        -6.6092596 ,  -6.6986594 ,  -6.7685156 ,  -6.827608  ,
        -6.876064  ,  -6.9170685 ,  -6.9516654 ,  -6.989456  ,
        -7.01795   ,  -7.046195  ,  -7.0705137 ,  -7.0921965 ,
        -7.115857  ,  -7.1366167 ,  -7.156224  ,  -7.1740417 ,
        -7.192025  ,  -7.2076263 ,  -7.2281303 ,  -7.245305  ,
        -7.2621098 ,  -7.2812333 ,  -7.3032007 ,  -7.322002  ,
        -7.343897  ,  -7.367383  ,  -7.390361  ,  -7.4170194 ,
        -7.4472065 ,  -7.4836655 ,  -8.657236  ,  -8.674448  ,
        -8.743008  ,  -8.965656  ,  -9.333185  ,  -9.736067  ,
       -10.129051  , -10.4543915 , -10.709713  , -10.959314  ,
       -11.274353  , -11.30052   , -11.037741  , -10.690728  ,
       -10.410459  , -10.244119  , -10.121136  ,  -9.895524  ,
        -9.517585  ,  -9.141989  ,  -8.977821  ,  -8.891142  ,
        -8.715811  ,  -8.592821  ,  -8.487829  ,  -8.136506  ,
        -6.9807806 ,  -5.5162473 ,  -3.394826  ,  -1.8635192 ,
        -0.65395486,  -0.5861296 ,  -0.7379432 ,  -0.9459083 ,
        -1.024064  ,  -0.604812  ,  -0.14159486,   0.23958103,
         0.94500196,   1.7382505 ,   2.446838  ,   2.8695526 ,
         3.2584357 ,   3.7606041 ,   4.421138  ,   5.1645784 ,
         5.7594557 ,   6.117281  ,   6.2800636 ,   6.3043494 ,
         6.376761  ,   6.6256976 ,   7.0503254 ,   7.6205835 ,
         8.290421  ,   8.884375  ,   9.3339205 ,   9.701159  ,
        10.141434  ,  10.772724  ,  11.388084  ,  11.661543  ,
        11.655415  ,  11.507788  ,  11.283876  ,  11.090929  ,
        10.784716  ,  10.244596  ,   9.532963  ,   8.341295  ,
         6.5870676 ,   6.0974655 ,   5.2187395 ,   5.9411116 ],
      dtype=float32)
Coordinates:
    latlon_coord  object CoordPair(lat=55.006928, lon=13.154189)
    x_y           <U1 'x'
Dimensions without coordinates: bottom_top

In [164]:
ws = np.sqrt(u**2 + v**2)

alpha = np.log(ws[near_z[0]+1]/ws[near_z[0]])/np.log(fino_hgt[near_z[0]+1]/near_z[1])

target_ws = ws[near_z[0]]*(62/near_z[1])**alpha

In [161]:
near_z

(13,
 <xarray.DataArray 'height_agl' ()>
 array(61.278534, dtype=float32)
 Coordinates:
     XLONG         float32 13.15
     XLAT          float32 55.01
     XTIME         float32 0.0
     Time          datetime64[ns] 2020-10-03T12:00:00
     latlon_coord  object CoordPair(lat=55.006928, lon=13.154189)
 Attributes:
     FieldType:    104
     MemoryOrder:  XYZ
     description:  model height - [AGL] (mass grid)
     units:        m
     stagger:      
     coordinates:  XLONG XLAT XTIME
     projection:   LambertConformal(stand_lon=2.0, moad_cen_lat=54.40000534057...)

In [179]:
fino_hgt[:20]

<xarray.DataArray 'height_agl' (bottom_top: 20)>
array([ 2.2766914,  6.830686 , 11.385902 , 15.904047 , 20.423262 ,
       24.98232  , 29.543045 , 34.06667  , 38.59146  , 43.11795  ,
       47.646633 , 52.21474  , 56.745796 , 61.278534 , 65.81248  ,
       70.30889  , 74.84573  , 79.38429  , 83.92455  , 88.46653  ],
      dtype=float32)
Coordinates:
    XLONG         float32 13.15
    XLAT          float32 55.01
    XTIME         float32 0.0
    Time          datetime64[ns] 2020-10-03T12:00:00
    latlon_coord  object CoordPair(lat=55.006928, lon=13.154189)
Dimensions without coordinates: bottom_top
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  model height - [AGL] (mass grid)
    units:        m
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=2.0, moad_cen_lat=54.40000534057...

In [170]:
near_z[1]

<xarray.DataArray 'height_agl' ()>
array(61.278534, dtype=float32)
Coordinates:
    XLONG         float32 13.15
    XLAT          float32 55.01
    XTIME         float32 0.0
    Time          datetime64[ns] 2020-10-03T12:00:00
    latlon_coord  object CoordPair(lat=55.006928, lon=13.154189)
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  model height - [AGL] (mass grid)
    units:        m
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=2.0, moad_cen_lat=54.40000534057...

In [175]:
alpha = np.log(ws[near_z[0]+1]/ws[near_z[0]])/np.log(fino_hgt[near_z[0]+1]/near_z[1])

target_ws = ws[near_z[0]]*(62/near_z[1])**alpha

In [176]:
target_ws

<xarray.DataArray ()>
array(10.99843057)
Coordinates:
    latlon_coord  object CoordPair(lat=55.006928, lon=13.154189)
    XLONG         float32 13.15
    XLAT          float32 55.01
    XTIME         float32 0.0
    Time          datetime64[ns] 2020-10-03T12:00:00

In [177]:
ws

<xarray.DataArray (bottom_top: 108)>
array([ 8.613701 ,  9.516677 ,  9.939848 , 10.189192 , 10.360073 ,
       10.492402 , 10.593607 , 10.67887  , 10.749757 , 10.809993 ,
       10.860392 , 10.912618 , 10.950654 , 10.992897 , 11.026686 ,
       11.059288 , 11.088436 , 11.118629 , 11.145087 , 11.169106 ,
       11.194375 , 11.216601 , 11.2419195, 11.267626 , 11.289878 ,
       11.314967 , 11.34254  , 11.368932 , 11.396911 , 11.426782 ,
       11.455785 , 11.488682 , 11.534655 , 11.580611 , 13.170813 ,
       13.202879 , 13.290976 , 13.595582 , 14.102501 , 14.655963 ,
       15.185664 , 15.60622  , 15.919304 , 16.197546 , 16.484314 ,
       16.409758 , 16.014204 , 15.492064 , 15.016823 , 14.666934 ,
       14.354581 , 13.864165 , 13.108119 , 12.335691 , 11.900794 ,
       11.705923 , 11.592562 , 11.684929 , 11.984542 , 12.183351 ,
       11.838588 , 11.289703 , 10.940586 , 10.91764  , 10.885717 ,
       10.898351 , 10.967633 , 10.883234 , 10.9188795, 11.453733 ,
       12.1328945, 12.602833 , 13.013601 , 12.97325  , 12.833959 ,
       12.880729 , 13.156423 , 13.7739725, 14.8689375, 16.251833 ,
       17.289522 , 17.66209  , 17.70205  , 17.801239 , 18.166039 ,
       18.745926 , 19.427263 , 20.130405 , 20.763842 , 21.235657 ,
       21.614752 , 22.064579 , 22.802725 , 24.008436 , 25.701654 ,
       27.47683  , 29.00742  , 30.405529 , 31.63181  , 32.15371  ,
       31.754175 , 29.913649 , 26.816395 , 24.236084 , 22.00103  ,
       19.138147 , 15.789359 , 13.5061455], dtype=float32)
Coordinates:
    latlon_coord  object CoordPair(lat=55.006928, lon=13.154189)
Dimensions without coordinates: bottom_top

In [ ]:
alpha = np.log(x_df[target_var+'_'+str(int(levs[1]))+'m']/x_df[target_var+'_'+str(int(levs[0]))+'m'])\
                /np.log(levs[1]/levs[0])

x_df[target_var+'_'+str(int(target_hgt))+'m'] = x_df[target_var+'_'+str(int(levs[1]))+'m']\
                                           *(target_hgt/levs[1])**alpha

In [3]:
glob.glob(file_dir+'wrfout_d03*')

[]

In [3]:
file_dir

'/projects/wfip2les/cdraxl/2020100300/'

In [11]:
glob.glob('/projects/wfip2les/cdraxl/2020100300/*')

[]

In [19]:
file_list = glob.glob(file_dir+'custom_wrfout_d03*')

In [21]:
len(file_list)

2160

In [5]:
glob.glob('/home/jlee/*')

['/home/jlee/era5',
 '/home/jlee/pkgs',
 '/home/jlee/wrf_bash',
 '/home/jlee/wfip',
 '/home/jlee/build_wrf',
 '/home/jlee/home_wrf',
 '/home/jlee/WW',
 '/home/jlee/mmcpnnl',
 '/home/jlee/rsync_script',
 '/home/jlee/code_py',
 '/home/jlee/awaken',
 '/home/jlee/utils.py',
 '/home/jlee/bash_pg',
 '/home/jlee/amrw']

In [5]:
ds = xr.open_dataset('/projects/wfip2les/cdraxl/2020100300/custom_wrfout_d03_2020-10-10_19_25_00.nc')

PermissionError: [Errno 13] Permission denied: b'/projects/wfip2les/cdraxl/2020100300/custom_wrfout_d03_2020-10-10_19_25_00.nc'